# Configuring `Users` and `Workspaces`

This tutorial is part of a series in which we will get to know the `FeedbackDataset`. In this step, we will show how to configure `Users` and `Workspaces`. If you need additional context, consult [user management](../../../../getting_started/installation/configurations/user_management.md) and [workspace management](../../../../getting_started/installation/configurations/workspace_management.md).

![workflow](../../../../_static/tutorials/end2end/base/workflow_users_and_workspaces.png)

## Table of Contents

1. [Configure a FeedbackDataset](#create-feedbackdataset)
2. [Add records](#add-records)
   1. [From a Hugging Face dataset](#create-feedbackdataset-datasets)
   2. [From a pandas.DataFrame](#create-feedbackdataset-pandas)
3. [Save a FeedbackDataset](#save-the-dataset)
   1. [Push our FeedbackDataset to Argilla](#push-to-argilla)
   2. [Push our FeedbackDataset to the Hugging Face hub](#push-to-hf-hub)
4. [Conclusions](#conclusions)

## Running Argilla

For this tutorial, you will need to have an Argilla server running. There are two main options for deploying and running Argilla:

**Deploy Argilla on Hugging Face Spaces:** If you want to run tutorials with external notebooks (e.g., Google Colab) and you have an account on Hugging Face, you can deploy Argilla on Spaces with a few clicks:

[![deploy on spaces](https://huggingface.co/datasets/huggingface/badges/raw/main/deploy-to-spaces-lg.svg)](https://huggingface.co/new-space?template=argilla/argilla-template-space)

For details about configuring your deployment, check the [official Hugging Face Hub guide](https://huggingface.co/docs/hub/spaces-sdks-docker-argilla).

**Launch Argilla using Argilla's quickstart Docker image**: This is the recommended option if you want [Argilla running on your local machine](../../../../getting_started/quickstart.md). Note that this option will only let you run the tutorial locally and not with an external notebook service.

For more information on deployment options, please check the Deployment section of the documentation.

<div class="alert alert-info">

Tip

This tutorial is a Jupyter Notebook. There are two options to run it:

- Use the Open in Colab button at the top of this page. This option allows you to run the notebook directly on Google Colab. Don't forget to change the runtime type to GPU for faster model training and inference.
- Download the .ipynb file by clicking on the View source link at the top of the page. This option allows you to download the notebook and run it on your local machine or on a Jupyter notebook tool of your choice.
</div>

First let's install our dependencies and import the necessary libraries:

In [38]:
!pip install "argilla[server]"

Looking in indexes: https://pypi.org/simple, https://dmrepository.datamaran.com:8443/repository/dmPYTHON/simple
  Using cached https://dmrepository.datamaran.com:8443/repository/dmPYTHON/packages/opensearch-py/2.0.1/opensearch_py-2.0.1-py2.py3-none-any.whl (214 kB)
  Using cached https://dmrepository.datamaran.com:8443/repository/dmPYTHON/packages/python-multipart/0.0.6/python_multipart-0.0.6-py3-none-any.whl (45 kB)
  Using cached https://dmrepository.datamaran.com:8443/repository/dmPYTHON/packages/alembic/1.9.4/alembic-1.9.4-py3-none-any.whl (210 kB)
  Using cached https://dmrepository.datamaran.com:8443/repository/dmPYTHON/packages/segment-analytics-python/2.2.0/segment_analytics_python-2.2.0-py2.py3-none-any.whl (23 kB)
  Using cached https://dmrepository.datamaran.com:8443/repository/dmPYTHON/packages/smart-open/6.4.0/smart_open-6.4.0-py3-none-any.whl (57 kB)
  Using cached https://dmrepository.datamaran.com:8443/repository/dmPYTHON/packages/aiofiles/22.1.0/aiofiles-22.1.0-py3-non

In [9]:
import argilla as rg
from argilla._constants import DEFAULT_API_KEY

In order to run this notebook we will need some credentials to push and load datasets from `Argilla` and 🤗`hub`, let's set them in the following cell:

In [54]:
# Argilla credentials
api_url = "http://localhost:6900" # "https://<YOUR-HF-SPACE>.hf.space"
api_key = DEFAULT_API_KEY # admin.apikey

Log to argilla:

In [11]:
rg.init(
    api_url=api_url,
    api_key=api_key
)

/Users/davidberenstein/Documents/programming/argilla/argilla/.venv/lib/python3.10/site-packages/argilla/client/client.py:124: UserWarning: Default user was detected and no workspace configuration was provided, so the default 'argilla' workspace will be used. If you want to setup another workspace, use the `rg.set_workspace` function or provide a different one on `rg.init`
  warnings.warn(
/Users/davidberenstein/Documents/programming/argilla/argilla/.venv/lib/python3.10/site-packages/argilla/client/client.py:152: UserWarning: You're connecting to Argilla Server 1.19.0 using a different client version (1.14.0).
This may lead to potential compatibility issues during your experience.
To ensure a seamless and optimized connection, we highly recommend aligning your client version with the server version.
  warnings.warn(


Alternatively, we can use the `CLI` to run the `argilla login` command and pass the `api_url` and `api_key` there.

In [12]:
!python -m argilla login --api-url {api_url} --api-key {api_key}

╭─ Logged in ──────────────────────────────────────────────────────────────────╮
│ Logged in successfully to 'http://localhost:6900' Argilla server!            │
╰──────────────────────────────────────────────────────────────────────────────╯


### Enable Telemetry

We gain valuable insights from how you interact with our tutorials. To improve ourselves in offering you the most suitable content, using the following lines of code will help us understand that this tutorial is serving you effectively. Though this is entirely anonymous, you can choose to skip this step if you prefer. For more info, please check out the [Telemetry](../../reference/telemetry.md) page.

In [ ]:
from argilla.utils.telemetry import tutorial_running

tutorial_running()

## Configure `Users`

### Current active `User`

For this tutorial, we'll start with exploring the currently active user we've configured during the initialization. We can do this using our `Python` client or `CLI`.

In [14]:
user = rg.User.me()
user

User(id=f3cc0d01-5129-41a6-a722-6c51b097abd0, username=argilla, role=owner, api_key=argilla.apikey, first_name=, last_name=None, inserted_at=2023-10-28 11:26:52.618074, updated_at=2023-10-28 11:26:52.618074)

In [13]:
!python -m argilla whoami

╭─ Current User ───────────────────────────────────────────────────────────────╮
│                                                                              │
│  • Username: argilla                                                         │
│  • Role: owner                                                               │
│  • First name:                                                               │
│  • Last name: None                                                           │
│  • API Key: argilla.apikey                                                   │
│  • Workspaces: ['argilla', 'awesome-argilla-datasets']                       │
╰──────────────────────────────────────────────────────────────────────────────╯


As we can see, we are logged into the default `User` called `argilla`, which has the `owner` role. This `role` is allowed to create new `Users` and configure workspaces.

### Create `User`

Next, we will create two new users with the role of `admin` and `annotator` to configure our first small team. For convenience and reproducibility, we will call them `admin` and `annotator`, and we'll set their `password` to the default `argilla.apikey`. Their `api_key` will be randomly generated according to safety standards.
We will first create the `admin` user with our `Python` client.

In [ ]:
user = rg.User.create(
    username="admin",
    password=api_key,
    role="admin",
)
user

In [ ]:
!python -m argilla users create --username owner --role owner --password {api_key}

### Update `User`

At the moment we can only update `User` role and this can only be done using the `CLI`.

In [55]:
!python -m argilla server database users update owner --role admin

User with username 'owner' does not exists in database. Skipping...


Now, let's quickly change this back to avoid having wrong access in our organization.

In [51]:
list(rg.User.list())

[User(id=f3cc0d01-5129-41a6-a722-6c51b097abd0, username=argilla, role=owner, api_key=argilla.apikey, first_name=, last_name=None, inserted_at=2023-10-28 11:26:52.618074, updated_at=2023-10-28 11:26:52.618074),
 User(id=e26a191f-382d-4fe8-9cb8-3d40e0588c20, username=jose, role=annotator, api_key=s2npW--dIQs1cLpTg4Tqd_zkWdzxu3WiuPkIcAKBo2q4z3iWWM9xPt2KDjBBr4fL3Cu5EBiED7n5mMdE5K7i4kRj126QN7T_F7QYyYQNh8A, first_name=jose, last_name=None, inserted_at=2023-11-19 15:50:37.630232, updated_at=2023-11-19 15:50:37.630232),
 User(id=bf9e8d61-f969-478f-8ea3-10f6f17119c6, username=dani, role=annotator, api_key=jMZWECjYgp4GHjcrpFOXSY1r6y4RSSRiXEGg7depEABCd18RanAmLdVk114MFuQxaHAi3d-TRgjr8OrH8pBYT54wqjCGUJD9Gn4UNa8PooE, first_name=dani, last_name=None, inserted_at=2023-11-19 15:52:56.040739, updated_at=2023-11-19 15:52:56.040739),
 User(id=4d067c1a-fbb7-4300-bf46-bc0f5e7954a5, username=paco, role=annotator, api_key=8Crwd_JGlvbM5UD7Og-vgBy5EyBE5aHLVXxt9yd-zdMnKHtITG6PzKV0ldBd_U0obOEOoUn6J36AVX-54FGofots

In [58]:
!python -m argilla server database users update owner --role owner

User with username 'owner' does not exists in database. Skipping...


## Conclusion

In this tutorial, we created some Argilla `Users` and `Workspaces`, and created some annotation team configurations.

If you need additional context, consult [user management](../../../../getting_started/installation/configurations/user_management.md) and [workspace management](../../../../getting_started/installation/configurations/workspace_management.md).

Alternatively, you can go to the next step and [create a FeedbackDataset](/tutorials_and_integrations/tutorials/feedback/end2end_examples/create-dataset.html).